In [15]:
import os
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import random
import sys
import cv2
import pickle as pkl
import tensorflow as tf
import tensorflow.keras.backend as K
from skimage.color import *
from skimage.morphology import *
from skimage.feature import *
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam, RMSprop 
from tensorflow.keras.initializers import he_uniform
# import tensorflow_addons as tfa
from tensorflow.keras.layers import *
from tqdm.notebook import tqdm
import imageio as io
import deepwatermap
import tifffile as tiff
from sklearn.metrics import average_precision_score

In [3]:
# Always run this otherwise TF crashes
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print('so')
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
def F1(y_true,y_pred):
    threshold = 0.5

    y_true = tf.reshape(y_true,[-1])
    y_pred = tf.reshape(y_pred,[-1])
    y_pred = (y_pred>threshold)
    y_pred = tf.cast(y_pred,dtype=tf.float32)

    tp = K.sum(y_true*(y_pred))
    tn = K.sum((1-y_true)*((1-y_pred)))
    fp = K.sum((1-y_true)*(y_pred))
    fn = K.sum((y_true)*((1-y_pred)))

    pr  = ((tp+1.)/(tp+fp+1.))
    rec = ((tp+1.)/(tp+fn+1.))
    f1  = ((2*pr*rec)/(pr+rec))
    return f1

In [5]:
def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    
    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) *     (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)

def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss

def loss_kaggle(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    averaged_mask = K.pool2d(
            y_true, pool_size=(11, 11), strides=(1, 1), padding='same', pool_mode='avg', data_format="channels_last")
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    
    loss = weighted_bce_loss(y_true, y_pred, weight) +\
    weighted_dice_loss(y_true, y_pred, weight) +\
    IoU_loss(y_true, y_pred, weight)
    
    return loss

def IoU(y_true, y_pred, weight):
    weight = weight*weight
    intersection = y_true*y_pred
    not_true     = 1 - y_true
    union        = y_true + (not_true * y_pred)
    score        = (K.sum(weight * intersection)) / (K.sum(weight * union))
    return score


# IoU as a loss function
def IoU_loss(y_true, y_pred, weight):
    return 1 - IoU(y_true, y_pred, weight)

In [6]:
################ Metrics #######################
def IoU_pr_rec_f1(y_true, y_pred):
    
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    y_pred = ((y_pred)*1.).astype('float32')
    
    tp = np.sum(y_true*(y_pred))
    tn = np.sum((1-y_true)*((1-y_pred)))
    fp = np.sum((1-y_true)*(y_pred))
    fn = np.sum((y_true)*((1-y_pred)))
    
    pr  = (tp/(tp+fp))
    rec = (tp/(tp+fn))
    f1  = ((2*pr*rec)/(pr+rec))
    tnr = (tn/(tn+fp))
    fpr = (fp/(fp+tn))
    
    intersection = y_true*y_pred
    not_true     = 1 - y_true
    union        = y_true + (not_true * y_pred)
    iou         = (np.sum(intersection)) / (np.sum(union))
    
    return iou, pr, rec, f1, tnr, fpr

# Saving Metrics
def metrics():
    x = np.arange(0,1,0.05)
    IoU_      = []
    threshold = []
    precision = []
    recall    = []
    F_score   = []
    TNR       = []
    FPR       = []
    name_list = []

    dict_1 = {'Threshold': threshold,
              'Name':name_list,
              'IoU':IoU_,
              'Precision':precision,
              'Recall':recall,
              'F-Score':F_score,
              'True Negative Rate':TNR,
              'False Positive Rate':FPR}
    return dict_1

#calculate F-scores only
#update other parameters after best f-socore is found
#
def best_f_score(name, test_masks, predictions) :
    dict_1 = metrics()
    outer = 0
    best_f_score = 0
    x = 0       
    y = 1
    delta = 0.05
    while outer<3:
        z = np.arange(x, y, delta)         #
        for threshold in z:
            iou,precision,recall,f_score,tnr,fpr = IoU_pr_rec_f1((test_masks), (predictions>threshold))
            dict_1['IoU'].append(iou)
            dict_1['Threshold'].append(threshold)
            dict_1['Precision'].append(precision)
            dict_1['Recall'].append(recall)
            dict_1['F-Score'].append(f_score)
            dict_1['True Negative Rate'].append(tnr)
            dict_1['False Positive Rate'].append(fpr)
            dict_1['Name'].append(name)
            if f_score>best_f_score:
                best_f_score = f_score
                x = threshold
            else:
                pass
        x-=     delta
        y = x + delta
        delta*= 0.1
        outer+= 1
        
    return dict_1

dict_1 = metrics()


# Sentinel

In [8]:
np.random.seed(49)
pop = np.arange(5121)
sample_train = np.random.choice(pop, 4608, replace=False)
sample_test  = np.delete(pop, sample_train)
len(sample_train), len(sample_test)

(4608, 513)

In [10]:
X      = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Sentinel uint16 Data 0%water and 0%land exclude\X_train_sentinel_6_channles_5121.npy")
X_test = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Sentinel uint16 Data 0%water and 0%land exclude\X_test_sentinel_6_channles_2053.npy")
X_train= X[sample_train]
X_val  = X[sample_test]

Y_train         = (X_train[...,1]-X_train[...,3])/(X_train[...,1]+X_train[...,3])
Y_val           = (X_val[...,1]-X_val[...,3])/(X_val[...,1]+X_val[...,3])
Y_test          = (X_test[...,1]-X_test[...,3])/(X_test[...,1]+X_test[...,3])

Y_train         = ((Y_train<1.)*1).astype('float32')
Y_val           = ((Y_val<1.)*1).astype('float32')
Y_test          = ((Y_test<1.)*1).astype('float32')

X_train         = X_train[...,2::-1].copy()
X_val           = X_val[...,2::-1].copy()
X_test          = X_test[...,2::-1].copy()

X_train         = np.clip(X_train, 0, 3558) 
X_val           = np.clip(X_val, 0, 3558)
X_test          = np.clip(X_test, 0, 3558) 

X_train         = X_train - X_train.min(axis=(1,2), keepdims=True) 
X_val           = X_val  - X_val.min(axis=(1,2), keepdims=True)
X_test          = X_test - X_test.min(axis=(1,2), keepdims=True) 

X_train         = X_train / X_train.max(axis=(1,2), keepdims=True) 
X_val           = X_val  / X_val.max(axis=(1,2), keepdims=True)
X_test          = X_test / X_test.max(axis=(1,2), keepdims=True) 

In [11]:
################ Get Contours ##########################
Y_train = np.array([binary_dilation(binary_dilation(mask))-mask for mask in Y_train ], dtype='float64')[...,np.newaxis]
Y_val   = np.array([binary_dilation(binary_dilation(mask))-mask for mask in Y_val], dtype='float64')[...,np.newaxis]
Y_test  = np.array([binary_dilation(binary_dilation(mask))-mask for mask in Y_test], dtype='float64')[...,np.newaxis]

In [13]:
model = deepwatermap.model()
opt   = Adam(learning_rate=0.003)
model.compile(optimizer=opt, loss= loss_kaggle, metrics=[F1])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   12          ['input_1[0][0]']                
                                4)                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   16         ['conv2d[0][0]']                 
 alization)                     4)                                                            

In [14]:
history = model.fit(X_train,Y_train, validation_data=(X_val, Y_val), batch_size=32,epochs=50)

Epoch 1/50
144/144 [==============================] - 22s 57ms/step - loss: 2.1962 - F1: 0.2354 - val_loss: 2.0154 - val_F1: 0.4024
Epoch 2/50
144/144 [==============================] - 7s 52ms/step - loss: 1.9119 - F1: 0.4153 - val_loss: 1.8212 - val_F1: 0.4669
Epoch 3/50
144/144 [==============================] - 8s 52ms/step - loss: 1.7108 - F1: 0.5001 - val_loss: 1.6327 - val_F1: 0.4753
Epoch 4/50
144/144 [==============================] - 8s 52ms/step - loss: 1.5617 - F1: 0.5393 - val_loss: 1.4986 - val_F1: 0.5463
Epoch 5/50
144/144 [==============================] - 8s 52ms/step - loss: 1.4348 - F1: 0.5710 - val_loss: 1.3720 - val_F1: 0.5865
Epoch 6/50
144/144 [==============================] - 8s 52ms/step - loss: 1.3314 - F1: 0.5975 - val_loss: 1.2941 - val_F1: 0.5895
Epoch 7/50
144/144 [==============================] - 8s 53ms/step - loss: 1.2527 - F1: 0.6138 - val_loss: 1.2688 - val_F1: 0.5739
Epoch 8/50
144/144 [==============================] - 8s 53ms/step - loss: 1.1804 

In [15]:
model.save('Deepwatermap_github_sentinel_2dil-ndwi.hdf5')

C:\Users\HPCL\anaconda3\envs\torch_env\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [16]:
predictions = model.predict(X_test).squeeze()
dict_1 = best_f_score('DWM Trained BCE 50 Epochs Sentinel', Y_test, predictions)
df_rotate = pd.DataFrame(dict_1)
df_rotate = df_rotate.sort_values(by=['F-Score'], ascending=False)
df_rotate.to_csv(r'F-score DWM.csv', mode='a')
df_rotate.head()

,Threshold,Name,IoU,Precision,Recall,F-Score,True Negative Rate,False Positive Rate
40,0.0850,DWM Trained BCE 50 Epochs Sentinel,0.453175,0.647053,0.601980,0.623703,0.989133,0.010867
27,0.0850,DWM Trained BCE 50 Epochs Sentinel,0.453175,0.647053,0.601980,0.623703,0.989133,0.010867
39,0.0845,DWM Trained BCE 50 Epochs Sentinel,0.453160,0.646932,0.602058,0.623689,0.989126,0.010874
38,0.0840,DWM Trained BCE 50 Epochs Sentinel,0.453149,0.646820,0.602136,0.623679,0.989119,0.010881
28,0.0900,DWM Trained BCE 50 Epochs Sentinel,0.453149,0.648048,0.601075,0.623679,0.989196,0.010804


In [19]:
from sklearn.metrics import average_precision_score
average_precision_score(Y_test.reshape(-1), predictions.reshape(-1))

0.5805885332085843

# Landsat 

In [8]:
X_train = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Landsat 30m Resolution\X_train_4500_30m_res.npy")
X_val   = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Landsat 30m Resolution\X_val_500_30m_res.npy")
X_test  = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Landsat 30m Resolution\X_test_2000_30m_res.npy")

Y_train = (X_train[...,1]-X_train[...,3])/(X_train[...,1]+X_train[...,3])
Y_val   = (X_val[...,1]  -X_val[...,3])  /(X_val[...,1]+X_val[...,3])
Y_test  = (X_test[...,1] -X_test[...,3]) /(X_test[...,1]+X_test[...,3])

Y_train = ((Y_train<1.)*1).astype('float32')
Y_val   = ((Y_val<1.)*1).astype('float32')
Y_test  = ((Y_test<1.)*1).astype('float32')

X_train = X_train[...,2::-1].copy()
X_val   = X_val[...,2::-1].copy()
X_test  = X_test[...,2::-1].copy()

X_train = X_train - X_train.min(axis=(1,2), keepdims=True) 
X_val   = X_val  - X_val.min(axis=(1,2), keepdims=True)
X_test  = X_test - X_test.min(axis=(1,2), keepdims=True) 

X_train = X_train / X_train.max(axis=(1,2), keepdims=True) 
X_val   = X_val  / X_val.max(axis=(1,2), keepdims=True)
X_test  = X_test / X_test.max(axis=(1,2), keepdims=True) 

len(X_train), len(X_val), len(X_test), len(Y_train), len(Y_val), len(Y_test)

C:\Users\HPCL\AppData\Local\Temp/ipykernel_13760/890076540.py:5: RuntimeWarning: invalid value encountered in true_divide
  Y_train = (X_train[...,1]-X_train[...,3])/(X_train[...,1]+X_train[...,3])
C:\Users\HPCL\AppData\Local\Temp/ipykernel_13760/890076540.py:7: RuntimeWarning: invalid value encountered in true_divide
  Y_test  = (X_test[...,1] -X_test[...,3]) /(X_test[...,1]+X_test[...,3])


(4500, 500, 2000, 4500, 500, 2000)

In [9]:
################ Get Contours ##########################
Y_train = np.array([binary_dilation(mask)-mask for mask in Y_train ], dtype='float32')[...,np.newaxis]
Y_val   = np.array([binary_dilation(mask)-mask for mask in Y_val], dtype='float32')[...,np.newaxis]
Y_test  = np.array([binary_dilation(mask)-mask for mask in Y_test], dtype='float32')[...,np.newaxis]

In [10]:
def F1(y_true,y_pred):
    threshold = 0.5

    y_true = tf.reshape(y_true,[-1])
    y_pred = tf.reshape(y_pred,[-1])
    y_pred = (y_pred>threshold)
    y_pred = tf.cast(y_pred,dtype=tf.float32)

    tp = K.sum(y_true*(y_pred))
    tn = K.sum((1-y_true)*((1-y_pred)))
    fp = K.sum((1-y_true)*(y_pred))
    fn = K.sum((y_true)*((1-y_pred)))

    pr  = ((tp+1.)/(tp+fp+1.))
    rec = ((tp+1.)/(tp+fn+1.))
    f1  = ((2*pr*rec)/(pr+rec))
    return f1

In [11]:
def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    
    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) *     (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)

def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss

def loss_kaggle(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    averaged_mask = K.pool2d(
            y_true, pool_size=(11, 11), strides=(1, 1), padding='same', pool_mode='avg', data_format="channels_last")
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    
    loss = weighted_bce_loss(y_true, y_pred, weight) +\
    weighted_dice_loss(y_true, y_pred, weight) +\
    IoU_loss(y_true, y_pred, weight)
    
    return loss

def IoU(y_true, y_pred, weight):
    weight = weight*weight
    intersection = y_true*y_pred
    not_true     = 1 - y_true
    union        = y_true + (not_true * y_pred)
    score        = (K.sum(weight * intersection)) / (K.sum(weight * union))
    return score


# IoU as a loss function
def IoU_loss(y_true, y_pred, weight):
    return 1 - IoU(y_true, y_pred, weight)

In [12]:
model = deepwatermap.model()
opt   = Adam(learning_rate=0.003)
model.compile(optimizer=opt, loss= loss_kaggle, metrics=[F1])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   12          ['input_1[0][0]']                
                                4)                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   16         ['conv2d[0][0]']                 
 alization)                     4)                                                            

In [13]:
history = model.fit(X_train,Y_train, validation_data=(X_val, Y_val), batch_size=32,epochs=50)

Epoch 1/50
141/141 [==============================] - 19s 56ms/step - loss: 2.4264 - F1: 0.1217 - val_loss: 2.2154 - val_F1: 0.2629
Epoch 2/50
141/141 [==============================] - 7s 50ms/step - loss: 2.1863 - F1: 0.2768 - val_loss: 2.0833 - val_F1: 0.3605
Epoch 3/50
141/141 [==============================] - 7s 51ms/step - loss: 2.0086 - F1: 0.3687 - val_loss: 1.9922 - val_F1: 0.3910
Epoch 4/50
141/141 [==============================] - 7s 50ms/step - loss: 1.8636 - F1: 0.4130 - val_loss: 1.8240 - val_F1: 0.4871
Epoch 5/50
141/141 [==============================] - 7s 51ms/step - loss: 1.7335 - F1: 0.4513 - val_loss: 1.6873 - val_F1: 0.4430
Epoch 6/50
141/141 [==============================] - 7s 51ms/step - loss: 1.6183 - F1: 0.4833 - val_loss: 1.5610 - val_F1: 0.4498
Epoch 7/50
141/141 [==============================] - 7s 51ms/step - loss: 1.5173 - F1: 0.5075 - val_loss: 1.4923 - val_F1: 0.5319
Epoch 8/50
141/141 [==============================] - 7s 51ms/step - loss: 1.4337 

In [16]:
predictions = model.predict(X_test).squeeze()
dict_1 = best_f_score('DWM Landsat', Y_test, predictions)
df = pd.DataFrame(dict_1)
df = df.sort_values(by=['F-Score'], ascending=False)
AP = average_precision_score(Y_test.squeeze().reshape(-1), predictions.reshape(-1))
df.to_csv(r'F-score DWM.csv', mode='a')
df.head()

,Threshold,Name,IoU,Precision,Recall,F-Score,True Negative Rate,False Positive Rate
21,0.0050,DWM Landsat,0.387283,0.527954,0.592422,0.558333,0.994260,0.005740
39,0.0045,DWM Landsat,0.387265,0.525991,0.594870,0.558314,0.994191,0.005809
38,0.0040,DWM Landsat,0.387186,0.523639,0.597720,0.558232,0.994107,0.005892
37,0.0035,DWM Landsat,0.387035,0.520802,0.601096,0.558075,0.994006,0.005994
36,0.0030,DWM Landsat,0.386866,0.517317,0.605392,0.557900,0.993879,0.006121


In [17]:
average_precision_score(Y_test.squeeze().reshape(-1), predictions.reshape(-1))

0.43775317534327784

# Landsat + Sentinel

In [11]:
np.random.seed(49)
pop = np.arange(5121)
sample_train = np.random.choice(pop, 4608, replace=False)
sample_test  = np.delete(pop, sample_train)
len(sample_train), len(sample_test)

X      = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Sentinel uint16 Data 0%water and 0%land exclude\X_train_sentinel_6_channles_5121.npy")
X_test_S = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Sentinel uint16 Data 0%water and 0%land exclude\X_test_sentinel_6_channles_2053.npy")
X_train_S= X[sample_train]
X_val_S  = X[sample_test]

Y_train_S         = (X_train_S[...,1]-X_train_S[...,3])/(X_train_S[...,1]+X_train_S[...,3])
Y_val_S           = (X_val_S[...,1]-X_val_S[...,3])/(X_val_S[...,1]+X_val_S[...,3])
Y_test_S          = (X_test_S[...,1]-X_test_S[...,3])/(X_test_S[...,1]+X_test_S[...,3])

Y_train_S         = ((Y_train_S<1.)*1).astype('float32')
Y_val_S           = ((Y_val_S<1.)*1).astype('float32')
Y_test_S          = ((Y_test_S<1.)*1).astype('float32')

X_train_S         = X_train_S[...,2::-1].copy()
X_val_S           = X_val_S[...,2::-1].copy()
X_test_S          = X_test_S[...,2::-1].copy()

X_train_S         = np.clip(X_train_S, 0, 3558) 
X_val_S           = np.clip(X_val_S, 0, 3558)
X_test_S          = np.clip(X_test_S, 0, 3558) 

X_train_S         = X_train_S - X_train_S.min(axis=(1,2), keepdims=True) 
X_val_S           = X_val_S   - X_val_S.min(axis=(1,2), keepdims=True)
X_test_S          = X_test_S  - X_test_S.min(axis=(1,2), keepdims=True) 

X_train_S         = X_train_S / X_train_S.max(axis=(1,2), keepdims=True) 
X_val_S           = X_val_S   / X_val_S.max(axis=(1,2), keepdims=True)
X_test_S          = X_test_S  / X_test_S.max(axis=(1,2), keepdims=True) 

X_train_L = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Landsat 30m Resolution\X_train_4500_30m_res.npy")
X_val_L   = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Landsat 30m Resolution\X_val_500_30m_res.npy")
X_test_L  = np.load(r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Paper Experiments\Data\Landsat 30m Resolution\X_test_2000_30m_res.npy")

Y_train_L         = (X_train_L[...,1]-X_train_L[...,3])/(X_train_L[...,1]+X_train_L[...,3])
Y_val_L           = (X_val_L[...,1]-X_val_L[...,3])/(X_val_L[...,1]+X_val_L[...,3])
Y_test_L          = (X_test_L[...,1]-X_test_L[...,3])/(X_test_L[...,1]+X_test_L[...,3])

Y_train_L         = ((Y_train_L<1.)*1).astype('float32')
Y_val_L           = ((Y_val_L<1.)*1).astype('float32')
Y_test_L          = ((Y_test_L<1.)*1).astype('float32')

X_train_L         = X_train_L[...,2::-1].copy()
X_val_L           = X_val_L[...,2::-1].copy()
X_test_L          = X_test_L[...,2::-1].copy()

X_train_L         = X_train_L - X_train_L.min(axis=(1,2), keepdims=True) 
X_val_L           = X_val_L  - X_val_L.min(axis=(1,2), keepdims=True)
X_test_L          = X_test_L - X_test_L.min(axis=(1,2), keepdims=True) 

X_train_L         = X_train_L / X_train_L.max(axis=(1,2), keepdims=True) 
X_val_L           = X_val_L  / X_val_L.max(axis=(1,2), keepdims=True)
X_test_L          = X_test_L / X_test_L.max(axis=(1,2), keepdims=True) 

X_train = np.concatenate((X_train_L, X_train_S), axis=0)
X_val   = np.concatenate((X_val_L, X_val_S), axis=0)
X_test  = np.concatenate((X_test_L, X_test_S), axis=0)

Y_train = np.concatenate((Y_train_L, Y_train_S), axis=0)
Y_val   = np.concatenate((Y_val_L, Y_val_S), axis=0)
Y_test  = np.concatenate((Y_test_L, Y_test_S), axis=0)

len(X_train), len(X_val), len(X_test), len(Y_train), len(Y_val), len(Y_test)

C:\Users\HPCL\AppData\Local\Temp/ipykernel_12952/466507815.py:40: RuntimeWarning: invalid value encountered in true_divide
  Y_train_L         = (X_train_L[...,1]-X_train_L[...,3])/(X_train_L[...,1]+X_train_L[...,3])
C:\Users\HPCL\AppData\Local\Temp/ipykernel_12952/466507815.py:42: RuntimeWarning: invalid value encountered in true_divide
  Y_test_L          = (X_test_L[...,1]-X_test_L[...,3])/(X_test_L[...,1]+X_test_L[...,3])


(9108, 1013, 4053, 9108, 1013, 4053)

In [12]:
################ Get Contours ##########################
Y_train = np.array([binary_dilation(mask)-mask for mask in Y_train ], dtype='float32')[...,np.newaxis]
Y_val   = np.array([binary_dilation(mask)-mask for mask in Y_val], dtype='float32')[...,np.newaxis]
Y_test  = np.array([binary_dilation(mask)-mask for mask in Y_test], dtype='float32')[...,np.newaxis]

Y_train_L = np.array([binary_dilation(mask)-mask for mask in Y_train_L ], dtype='float32')[...,np.newaxis]
Y_val_L   = np.array([binary_dilation(mask)-mask for mask in Y_val_L], dtype='float32')[...,np.newaxis]
Y_test_L  = np.array([binary_dilation(mask)-mask for mask in Y_test_L], dtype='float32')[...,np.newaxis]

Y_train_S = np.array([binary_dilation(mask)-mask for mask in Y_train_S], dtype='float32')[...,np.newaxis]
Y_val_S   = np.array([binary_dilation(mask)-mask for mask in Y_val_S], dtype='float32')[...,np.newaxis]
Y_test_S  = np.array([binary_dilation(mask)-mask for mask in Y_test_S], dtype='float32')[...,np.newaxis]

In [18]:
def F1(y_true,y_pred):
    threshold = 0.5

    y_true = tf.reshape(y_true,[-1])
    y_pred = tf.reshape(y_pred,[-1])
    y_pred = (y_pred>threshold)
    y_pred = tf.cast(y_pred,dtype=tf.float32)

    tp = K.sum(y_true*(y_pred))
    tn = K.sum((1-y_true)*((1-y_pred)))
    fp = K.sum((1-y_true)*(y_pred))
    fn = K.sum((y_true)*((1-y_pred)))

    pr  = ((tp+1.)/(tp+fp+1.))
    rec = ((tp+1.)/(tp+fn+1.))
    f1  = ((2*pr*rec)/(pr+rec))
    return f1

In [19]:
def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    
    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) *     (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)

def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss

def loss_kaggle(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    averaged_mask = K.pool2d(
            y_true, pool_size=(11, 11), strides=(1, 1), padding='same', pool_mode='avg', data_format="channels_last")
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    
    loss = weighted_bce_loss(y_true, y_pred, weight) +\
    weighted_dice_loss(y_true, y_pred, weight) +\
    IoU_loss(y_true, y_pred, weight)
    
    return loss

def IoU(y_true, y_pred, weight):
    weight = weight*weight
    intersection = y_true*y_pred
    not_true     = 1 - y_true
    union        = y_true + (not_true * y_pred)
    score        = (K.sum(weight * intersection)) / (K.sum(weight * union))
    return score


# IoU as a loss function
def IoU_loss(y_true, y_pred, weight):
    return 1 - IoU(y_true, y_pred, weight)

In [20]:
model = deepwatermap.model()
opt   = Adam(learning_rate=0.003)
model.compile(optimizer=opt, loss= loss_kaggle, metrics=[F1])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_20 (Conv2D)             (None, None, None,   12          ['input_2[0][0]']                
                                4)                                                                
                                                                                                  
 batch_normalization_20 (BatchN  (None, None, None,   16         ['conv2d_20[0][0]']              
 ormalization)                  4)                                                          

In [21]:
history = model.fit(X_train,Y_train, validation_data=(X_val, Y_val), batch_size=32,epochs=50)

Epoch 1/50
285/285 [==============================] - 31s 54ms/step - loss: 2.3314 - F1: 0.1547 - val_loss: 2.0838 - val_F1: 0.3087
Epoch 2/50
285/285 [==============================] - 15s 51ms/step - loss: 1.9650 - F1: 0.3399 - val_loss: 1.8376 - val_F1: 0.4008
Epoch 3/50
285/285 [==============================] - 14s 51ms/step - loss: 1.7212 - F1: 0.4179 - val_loss: 1.6166 - val_F1: 0.4767
Epoch 4/50
285/285 [==============================] - 14s 51ms/step - loss: 1.5457 - F1: 0.4609 - val_loss: 1.5984 - val_F1: 0.3946
Epoch 5/50
285/285 [==============================] - 15s 52ms/step - loss: 1.4214 - F1: 0.4902 - val_loss: 1.3783 - val_F1: 0.4958
Epoch 6/50
285/285 [==============================] - 15s 52ms/step - loss: 1.3376 - F1: 0.5079 - val_loss: 1.2939 - val_F1: 0.5190
Epoch 7/50
285/285 [==============================] - 15s 52ms/step - loss: 1.2748 - F1: 0.5238 - val_loss: 1.2552 - val_F1: 0.5271
Epoch 8/50
285/285 [==============================] - 15s 52ms/step - loss: 

In [22]:
model.save('Deepwatermap_github_landsat+sentinel.hdf5')

C:\Users\HPCL\anaconda3\envs\torch_env\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [23]:
################ Metrics #######################
def IoU_pr_rec_f1(y_true, y_pred):
    
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    y_pred = ((y_pred)*1.).astype('float32')
    
    tp = np.sum(y_true*(y_pred))
    tn = np.sum((1-y_true)*((1-y_pred)))
    fp = np.sum((1-y_true)*(y_pred))
    fn = np.sum((y_true)*((1-y_pred)))
    
    pr  = (tp/(tp+fp))
    rec = (tp/(tp+fn))
    f1  = ((2*pr*rec)/(pr+rec))
    tnr = (tn/(tn+fp))
    fpr = (fp/(fp+tn))
    
    intersection = y_true*y_pred
    not_true     = 1 - y_true
    union        = y_true + (not_true * y_pred)
    iou         = (np.sum(intersection)) / (np.sum(union))
    
    return iou, pr, rec, f1, tnr, fpr

# Saving Metrics
def metrics():
    x = np.arange(0,1,0.05)
    IoU_      = []
    threshold = []
    precision = []
    recall    = []
    F_score   = []
    TNR       = []
    FPR       = []
    name_list = []

    dict_1 = {'Threshold': threshold,
              'Name':name_list,
              'IoU':IoU_,
              'Precision':precision,
              'Recall':recall,
              'F-Score':F_score,
              'True Negative Rate':TNR,
              'False Positive Rate':FPR}
    return dict_1

#calculate F-scores only
#update other parameters after best f-socore is found
#
def best_f_score(name, test_masks, predictions) :
    dict_1 = metrics()
    outer = 0
    best_f_score = 0
    x = 0       
    y = 1
    delta = 0.05
    while outer<3:
        z = np.arange(x, y, delta)         #
        for threshold in z:
            iou,precision,recall,f_score,tnr,fpr = IoU_pr_rec_f1((test_masks), (predictions>threshold))
            dict_1['IoU'].append(iou)
            dict_1['Threshold'].append(threshold)
            dict_1['Precision'].append(precision)
            dict_1['Recall'].append(recall)
            dict_1['F-Score'].append(f_score)
            dict_1['True Negative Rate'].append(tnr)
            dict_1['False Positive Rate'].append(fpr)
            dict_1['Name'].append(name)
            if f_score>best_f_score:
                best_f_score = f_score
                x = threshold
            else:
                pass
        x-=     delta
        y = x + delta
        delta*= 0.1
        outer+= 1
        
    return dict_1

dict_1 = metrics()


In [25]:
predictions = model.predict(X_test).squeeze()
dict_1 = best_f_score('DWM Trained BCE 50 Epochs Landsat+Sentinel', Y_test, predictions)
df_rotate = pd.DataFrame(dict_1)
df_rotate = df_rotate.sort_values(by=['F-Score'], ascending=False)
AP = average_precision_score(Y_test.squeeze().reshape(-1), predictions.reshape(-1))
df_rotate.to_csv(r'F-score DWM.csv', mode='a')
df_rotate.head()

,Threshold,Name,IoU,Precision,Recall,F-Score,True Negative Rate,False Positive Rate
40,0.0900,DWM Trained BCE 50 Epochs Landsat+Sentinel,0.348317,0.503058,0.531037,0.516669,0.992747,0.007253
28,0.0900,DWM Trained BCE 50 Epochs Landsat+Sentinel,0.348317,0.503058,0.531037,0.516669,0.992747,0.007253
2,0.1000,DWM Trained BCE 50 Epochs Landsat+Sentinel,0.348315,0.504736,0.529175,0.516667,0.992821,0.007179
39,0.0895,DWM Trained BCE 50 Epochs Landsat+Sentinel,0.348313,0.502967,0.531130,0.516665,0.992743,0.007257
37,0.0885,DWM Trained BCE 50 Epochs Landsat+Sentinel,0.348307,0.502778,0.531325,0.516658,0.992735,0.007265


In [31]:
predictions = model.predict(X_test_L).squeeze()
dict_1 = best_f_score('DWM Trained BCE 50 Epochs Landsat', Y_test_L, predictions)
df_rotate = pd.DataFrame(dict_1)
df_rotate = df_rotate.sort_values(by=['F-Score'], ascending=False)
AP = average_precision_score(Y_test_L.squeeze().reshape(-1), predictions.reshape(-1))
# df_rotate.to_csv(r'F-score DWM.csv', mode='a')
df_rotate.head()

,Threshold,Name,IoU,Precision,Recall,F-Score,True Negative Rate,False Positive Rate
35,0.1325,DWM Trained BCE 50 Epochs Landsat,0.382093,0.523876,0.585371,0.552919,0.994235,0.005765
34,0.1320,DWM Trained BCE 50 Epochs Landsat,0.382085,0.523812,0.585434,0.552911,0.994233,0.005767
36,0.1330,DWM Trained BCE 50 Epochs Landsat,0.382083,0.523926,0.585286,0.552909,0.994237,0.005763
38,0.1340,DWM Trained BCE 50 Epochs Landsat,0.382074,0.524032,0.585132,0.552899,0.994241,0.005759
39,0.1345,DWM Trained BCE 50 Epochs Landsat,0.382073,0.524086,0.585064,0.552898,0.994243,0.005757


In [32]:
AP

0.47970960266633156

In [33]:
predictions = model.predict(X_test_S).squeeze()
dict_1 = best_f_score('DWM Trained BCE 50 Epochs Sentinel', Y_test_S, predictions)
df_rotate = pd.DataFrame(dict_1)
df_rotate = df_rotate.sort_values(by=['F-Score'], ascending=False)
AP = average_precision_score(Y_test_S.squeeze().reshape(-1), predictions.reshape(-1))
# df_rotate.to_csv(r'F-score DWM.csv', mode='a')
df_rotate.head()

,Threshold,Name,IoU,Precision,Recall,F-Score,True Negative Rate,False Positive Rate
40,0.0700,DWM Trained BCE 50 Epochs Sentinel,0.326485,0.488017,0.496569,0.492256,0.991271,0.008729
24,0.0700,DWM Trained BCE 50 Epochs Sentinel,0.326485,0.488017,0.496569,0.492256,0.991271,0.008729
39,0.0695,DWM Trained BCE 50 Epochs Sentinel,0.326483,0.487881,0.496706,0.492254,0.991264,0.008736
25,0.0750,DWM Trained BCE 50 Epochs Sentinel,0.326480,0.489168,0.495371,0.492250,0.991332,0.008668
37,0.0685,DWM Trained BCE 50 Epochs Sentinel,0.326453,0.487622,0.496904,0.492219,0.991251,0.008749


In [34]:
AP

0.41468884418691176

In [27]:
predictions.shape

(4053, 128, 128)

In [29]:
Y_test.dtype

dtype('float32')

0.44037982422109323